In [1]:
import numpy as np
import pandas as pd

In [6]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.regularizers import l2

import os
import PIL

In [9]:
df = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')
df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,...,pixel744,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [11]:
y =df['label']

data=df.drop(columns = 'label')
print(data)

       pixel0  pixel1  pixel2  pixel3  ...  pixel780  pixel781  pixel782  pixel783
0           0       0       0       0  ...         0         0         0         0
1           0       0       0       0  ...         0         0         0         0
2           0       0       0       0  ...         0         0         0         0
3           0       0       0       0  ...         0         0         0         0
4           0       0       0       0  ...         0         0         0         0
...       ...     ...     ...     ...  ...       ...       ...       ...       ...
41995       0       0       0       0  ...         0         0         0         0
41996       0       0       0       0  ...         0         0         0         0
41997       0       0       0       0  ...         0         0         0         0
41998       0       0       0       0  ...         0         0         0         0
41999       0       0       0       0  ...         0         0         0         0

[42

In [12]:
data = np.array(data, dtype="float32")
data = data/255.0
img_height, img_width = 28,28

X = data.reshape(-1, img_height, img_width, 1)
# print(X)

y = to_categorical(y, num_classes=10)

In [13]:
seed =100
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.3 ,random_state = seed)

In [14]:
LR = ReduceLROnPlateau(monitor='val_loss', factor=0.2, verbose=1, patience=3,mode='min', epsilon=0.0001, cooldown=0, min_lr=0.00001)
ES = EarlyStopping(monitor='val_loss', patience=5, verbose=1, mode='min')
callbacks_list = [LR,ES]

In [15]:
model = Sequential([
    # 1st conv layer
  layers.Conv2D(32, 3, padding='same', activation='relu',input_shape=(img_height,img_width,1)),
  layers.BatchNormalization(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
    # maxpooling layer
  layers.MaxPooling2D(),
    # 2nd conv layer
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.BatchNormalization(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
    # maxpooling layer
  layers.MaxPooling2D(),
    # 3rd conv layer
  layers.Conv2D(128, 3, padding='same', activation='relu'),
  layers.BatchNormalization(),
  layers.Conv2D(128, 3, padding='same', activation='relu'),
     # maxpooling layer
  layers.MaxPooling2D(),
     # flatten
  layers.Flatten(),
  layers.Dropout(0.25),
  layers.Dense(256, activation='relu'),
  layers.Dropout(0.25), 
  layers.Dense(128, activation='relu'),
  layers.Dropout(0.25),
  layers.Dense(10, activation='softmax')
])

In [16]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [17]:
epochs = 40
history = model.fit(X_train, y_train, batch_size=32, validation_data=(X_test, y_test),
  steps_per_epoch = X_train.shape[0]/32,
  callbacks = callbacks_list,
  epochs=epochs
)

Epoch 1/50
393/393 [==============================] - 139s 348ms/step - loss: 0.4288 - accuracy: 0.8691 - val_loss: 1.1335 - val_accuracy: 0.6456 - lr: 0.0010
Epoch 2/50
393/393 [==============================] - 118s 300ms/step - loss: 0.1279 - accuracy: 0.9649 - val_loss: 0.1132 - val_accuracy: 0.9667 - lr: 0.0010
Epoch 3/50
393/393 [==============================] - 117s 297ms/step - loss: 0.0948 - accuracy: 0.9744 - val_loss: 0.0798 - val_accuracy: 0.9783 - lr: 0.0010
Epoch 4/50
393/393 [==============================] - 116s 295ms/step - loss: 0.0747 - accuracy: 0.9785 - val_loss: 0.0784 - val_accuracy: 0.9778 - lr: 0.0010
Epoch 5/50
393/393 [==============================] - 116s 296ms/step - loss: 0.0686 - accuracy: 0.9802 - val_loss: 0.0755 - val_accuracy: 0.9796 - lr: 0.0010
Epoch 6/50
393/393 [==============================] - 116s 295ms/step - loss: 0.0580 - accuracy: 0.9843 - val_loss: 0.0760 - val_accuracy: 0.9797 - lr: 0.0010
Epoch 7/50
393/393 [==========================

In [ ]:
test = np.array(df_test, dtype=np.float32)/255
test = test.reshape(-1,28,28,1)
prediction = model.predict(test)
predict = np.array(np.round(prediction), dtype = np.int32)
predict = np.argmax(predict , axis=1).reshape(-1, 1)
submission_df = pd.read_csv("sample_submission.csv")
submission_df['Label'] = predict
submission_df.to_csv('submission.csv', index=False)